# 데이터 테이블 통합
* 불러온 데이터 테이블 개수 : 8
* 통합한 데이터 테이블 개수 : 7

In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_info_columns', 150)

# csv 파일 불러오기

* 날짜를 datetime 포맷으로
* zip code는 문자열로

In [4]:
dir = "./data_version2/"

geolocation = pd.read_csv(dir + 'geolocation_olist_public_dataset.csv', dtype = {'zip_code_prefix' : str})
classified = pd.read_csv(dir + 'olist_classified_public_dataset.csv', dtype = {'customer_zip_code_prefix' : str},
                         parse_dates = ['order_purchase_timestamp', 'order_aproved_at', 'order_estimated_delivery_date', 'order_delivered_customer_date', 'review_creation_date', 'review_answer_timestamp'])
dataset = pd.read_csv(dir + 'olist_public_dataset_v2.csv', dtype = {'customer_zip_code_prefix' : str},
                      parse_dates = ['order_purchase_timestamp', 'order_aproved_at', 'order_estimated_delivery_date', 'order_delivered_customer_date', 'review_creation_date', 'review_answer_timestamp'])
customers = pd.read_csv(dir + 'olist_public_dataset_v2_customers.csv')
payments = pd.read_csv(dir + 'payments_olist_public_dataset.csv')
translation = pd.read_csv(dir + 'product_category_name_translation.csv')
product_measures = pd.read_csv(dir + 'product_measures_olist_public_dataset_.csv')
sellers = pd.read_csv(dir + 'sellers_olist_public_dataset_.csv', dtype = {'seller_zip_code_prefix' : str})

In [5]:
dataset.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2405
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_category_name                0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             91738
review_comment_message           57511
review_creation_date                 0
review_answer_timestamp              0
dtype: int64

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
order_id                         100000 non-null object
order_status                     100000 non-null object
order_products_value             100000 non-null float64
order_freight_value              100000 non-null float64
order_items_qty                  100000 non-null int64
order_sellers_qty                100000 non-null int64
order_purchase_timestamp         100000 non-null datetime64[ns]
order_aproved_at                 99982 non-null datetime64[ns]
order_estimated_delivery_date    100000 non-null datetime64[ns]
order_delivered_customer_date    97595 non-null datetime64[ns]
customer_id                      100000 non-null object
customer_city                    100000 non-null object
customer_state                   100000 non-null object
customer_zip_code_prefix         100000 non-null object
product_category_name            100000 non-null object
product_name_lenght 

# dataset preprocess : 카테고리명을 영어로(dataset->dataset_eng)

In [7]:
dataset.product_category_name

0                             beleza_saude
1                   informatica_acessorios
2                               automotivo
3                          cama_mesa_banho
4                         moveis_decoracao
5                            esporte_lazer
6                               perfumaria
7                    utilidades_domesticas
8                          cama_mesa_banho
9                         moveis_decoracao
10                        moveis_decoracao
11                               telefonia
12                  informatica_acessorios
13                      relogios_presentes
14                       alimentos_bebidas
15                           esporte_lazer
16                                   bebes
17                            beleza_saude
18                         cama_mesa_banho
19                               papelaria
20                        moveis_decoracao
21                               telefonia
22                tablets_impressao_imagem
23         

In [8]:
translation

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
5,esporte_lazer,sports_leisure
6,perfumaria,perfumery
7,utilidades_domesticas,housewares
8,telefonia,telephony
9,relogios_presentes,watches_gifts


In [9]:
dataset_eng = dataset.merge(translation, on='product_category_name').drop('product_category_name', axis=1)

In [10]:
dataset_eng.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2405
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             91738
review_comment_message           57511
review_creation_date                 0
review_answer_timestamp              0
product_category_name_english        0
dtype: int64

In [11]:
dataset_eng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 25 columns):
order_id                         100000 non-null object
order_status                     100000 non-null object
order_products_value             100000 non-null float64
order_freight_value              100000 non-null float64
order_items_qty                  100000 non-null int64
order_sellers_qty                100000 non-null int64
order_purchase_timestamp         100000 non-null datetime64[ns]
order_aproved_at                 99982 non-null datetime64[ns]
order_estimated_delivery_date    100000 non-null datetime64[ns]
order_delivered_customer_date    97595 non-null datetime64[ns]
customer_id                      100000 non-null object
customer_city                    100000 non-null object
customer_state                   100000 non-null object
customer_zip_code_prefix         100000 non-null object
product_name_lenght              100000 non-null int64
product_description_l

# payments(와이드포맷) 테이블과 join
## order_id가 여러 row에서 중복되는 경우 해당 order_id는 dataset에서 drop(dataset_eng->dataset2)

### order_id 집합별로 교집합 확인

In [12]:
# 중복되지 않은 order_id 집합 : 전체 order_it 집합 - 중복된 order_id 집합
pure_order_id_s = set(dataset_eng['order_id']) - set(dataset_eng[dataset_eng.duplicated('order_id', keep=False)]['order_id'])
print(len(pure_order_id_s))

# 중복된 order_id 집합
dirty_order_id_s = set(dataset_eng[dataset_eng.duplicated('order_id', keep=False)]['order_id'])
print(len(dirty_order_id_s))

# 집합이 잘 분리되었는지 확인
print(len(set(dataset_eng.order_id)))
print(len(pure_order_id_s) + len(dirty_order_id_s))

93003
3261
96264
96264


In [13]:
# payments에 있는 order_id 집합
payments_order_id_s = set(payments.order_id)
print(len(payments_order_id_s))

93000


In [14]:
# 3개 집합간 관계
print(pure_order_id_s & dirty_order_id_s)
print(len(pure_order_id_s & payments_order_id_s))
print(len(dirty_order_id_s & payments_order_id_s))

set()
93000
0


In [15]:
pure_order_id_s > payments_order_id_s

True

### dataset_eng - right join - payments

In [16]:
payments.nunique()

payments2 = payments.sort_values(by=['sequential','order_id'],ascending=True).reset_index(drop=True)
payments2

payments2['idx'] = payments2.groupby('order_id').cumcount()+1
payments2['install_idx'] = 'installments_' + payments2.idx.astype(str)
payments2['payment_type_idx'] = 'payment_type_' + payments2.idx.astype(str)
payments2['value_idx'] = 'value_' + payments2.idx.astype(str)
payments2

len_sequence = payments2.groupby('order_id').count()['sequential'].to_frame()
len_sequence.sort_values('sequential')

myorder = [0,11,22,23,24,25,26,27,28,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21]

installments = payments2.pivot(index='order_id', columns = 'install_idx',values='installments')
col_list = installments.columns.tolist()
col_list = [col_list[i] for i in myorder]
installments = installments[col_list]
installments

payment_type = payments2.pivot(index='order_id',columns='payment_type_idx',values='payment_type')
col_list = payment_type.columns.tolist()
col_list = [col_list[i] for i in myorder]
payment_type = payment_type[col_list]
payment_type

values = payments2.pivot(index='order_id',columns='value_idx',values='value')
col_list = values.columns.tolist()
col_list = [col_list[i] for i in myorder]
values = values[col_list]
values

wide_payments = pd.concat([installments,payment_type,values,len_sequence],axis=1)
wide_payments

wide_payments.reset_index(inplace=True)
wide_payments.head()

wide_payments.shape

(93000, 89)

In [17]:
wide_payments.isna().sum()

order_id               0
installments_1         0
installments_2     90132
installments_3     92417
installments_4     92706
installments_5     92823
installments_6     92882
installments_7     92917
installments_8     92941
installments_9     92950
installments_10    92960
installments_11    92967
installments_12    92976
installments_13    92983
installments_14    92987
installments_15    92990
installments_16    92992
installments_17    92992
installments_18    92993
installments_19    92993
installments_20    92995
installments_21    92995
installments_22    92996
installments_23    92996
installments_24    92996
installments_25    92996
installments_26    92996
installments_27    92998
installments_28    92999
installments_29    92999
payment_type_1         0
payment_type_2     90132
payment_type_3     92417
payment_type_4     92706
payment_type_5     92823
payment_type_6     92882
payment_type_7     92917
payment_type_8     92941
payment_type_9     92950
payment_type_10    92960


In [18]:
dataset2 = pd.merge(dataset_eng, wide_payments, how = 'left', on = 'order_id')

In [19]:
dataset2.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2405
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             91738
review_comment_message           57511
review_creation_date                 0
review_answer_timestamp              0
product_category_name_english        0
installments_1           

In [20]:
wide_payments.isna().sum()
dataset2.installments_1.isna().sum()
dataset2 = dataset2[~dataset2.installments_1.isna()]

In [21]:
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 99999
Data columns (total 113 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
customer_id                      93000 non-null object
customer_city                    93000 non-null object
customer_state                   93000 non-null object
customer_zip_code_prefix         93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       9

In [22]:
dataset2.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2295
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             85243
review_comment_message           54108
review_creation_date                 0
review_answer_timestamp              0
product_category_name_english        0
installments_1           

### dataset_eng에서 총 결제금액 ~ payments 총 결제금액 비교

In [23]:
dataset_compare = dataset2.copy()
dataset_compare

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_id,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name_english,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_13,installments_14,installments_15,installments_16,installments_17,installments_18,installments_19,installments_20,installments_21,installments_22,installments_23,installments_24,installments_25,installments_26,installments_27,installments_28,installments_29,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,payment_type_6,payment_type_7,payment_type_8,payment_type_9,payment_type_10,payment_type_11,payment_type_12,payment_type_13,payment_type_14,payment_type_15,payment_type_16,payment_type_17,payment_type_18,payment_type_19,payment_type_20,payment_type_21,payment_type_22,payment_type_23,payment_type_24,payment_type_25,payment_type_26,payment_type_27,payment_type_28,payment_type_29,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,value_16,value_17,value_18,value_19,value_20,value_21,value_22,value_23,value_24,value_25,value_26,value_27,value_28,value_29,sequential
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01,2017-02-01 02:41:21.549551,2017-03-15,2017-02-06 11:04:24.154259,297dcf281305aabfee256d313076af2b,Guaratuba,PR,832,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07,2017-02-09 02:37:37,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boleto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,363.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,8a723730400b508cbf47fbef4a76ec8e,delivered,60.00,20.91,1,1,2018-02-18 12:41:01,2018-02-18 13:46:36.606696,2018-03-14,2018-03-03 02:48:53.603256,f19376e0c54da9409485f7ef9556d367,Belo Horizonte,MG,308,38,1665,1,5858f45c20fde7d7e49af37a2166635a,8a723730400b508cbf47fbef4a76ec8e,5,NaN,muito bom cabelo fica lisinho,2018-03-04,2018-03-07 02:53:50,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit_card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,cf71b9b6b7ed12d72f541b0420023f19,delivered,79.99,10.94,1,1,2018-05-09 23:33:28,2018-05-11 02:57:52.916830,2018-05-24,2018-05-17 22:31:58.137648,12698ad9da6b62003811594de0ae297e,Cachoeiro de Itapemirim,ES,293,59,492,3,2b4609f8948be18874494203496bc318,cf71b9b6b7ed12d72f541b0420023f19,4,Bom,gostei,2018-05-18,2018-05-20 19:47:57,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boleto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,93ef9253c47e7dfc633c01f728488cb1,delivered,58.99,11.73,1,1,2017-10-26 09:07:04,2017-10-27 09:15:12.032978,2017-11-09,2017-10-31 16:09:47.272654,ba21875fac3c41b6ac433113cf862e55,Maua,SP,093,59,492,2,7c1bd920dbdf22470b68bde975dd3ccf,93ef

In [24]:
dataset_compare['payments_value_sum'] = dataset_compare.iloc[:,-30:-1].sum(axis = 1)
dataset_compare['order_total_value'] = dataset_compare['order_products_value'] + dataset_compare['order_freight_value']

In [25]:
dataset_compare

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_id,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name_english,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_13,installments_14,installments_15,installments_16,installments_17,installments_18,installments_19,installments_20,installments_21,installments_22,installments_23,installments_24,installments_25,installments_26,installments_27,installments_28,installments_29,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,payment_type_6,payment_type_7,payment_type_8,payment_type_9,payment_type_10,payment_type_11,payment_type_12,payment_type_13,payment_type_14,payment_type_15,payment_type_16,payment_type_17,payment_type_18,payment_type_19,payment_type_20,payment_type_21,payment_type_22,payment_type_23,payment_type_24,payment_type_25,payment_type_26,payment_type_27,payment_type_28,payment_type_29,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,value_16,value_17,value_18,value_19,value_20,value_21,value_22,value_23,value_24,value_25,value_26,value_27,value_28,value_29,sequential,payments_value_sum,order_total_value
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01,2017-02-01 02:41:21.549551,2017-03-15,2017-02-06 11:04:24.154259,297dcf281305aabfee256d313076af2b,Guaratuba,PR,832,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07,2017-02-09 02:37:37,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boleto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,363.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,363.74,363.74
2,8a723730400b508cbf47fbef4a76ec8e,delivered,60.00,20.91,1,1,2018-02-18 12:41:01,2018-02-18 13:46:36.606696,2018-03-14,2018-03-03 02:48:53.603256,f19376e0c54da9409485f7ef9556d367,Belo Horizonte,MG,308,38,1665,1,5858f45c20fde7d7e49af37a2166635a,8a723730400b508cbf47fbef4a76ec8e,5,NaN,muito bom cabelo fica lisinho,2018-03-04,2018-03-07 02:53:50,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit_card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,80.91,80.91
3,cf71b9b6b7ed12d72f541b0420023f19,delivered,79.99,10.94,1,1,2018-05-09 23:33:28,2018-05-11 02:57:52.916830,2018-05-24,2018-05-17 22:31:58.137648,12698ad9da6b62003811594de0ae297e,Cachoeiro de Itapemirim,ES,293,59,492,3,2b4609f8948be18874494203496bc318,cf71b9b6b7ed12d72f541b0420023f19,4,Bom,gostei,2018-05-18,2018-05-20 19:47:57,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boleto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,90.93,90.93
4,93ef9253c47e7dfc633c01f728488cb1,delivered,58.99,11.73,1,1,2017-10-26 09:07:04,2017-10-27 09:15:12.032978,2017-11-09,2017-10-31 16:09:47.272654,ba21875fac3c41b6

In [26]:
dataset_compare[~(dataset_compare['payments_value_sum'].round(2) == dataset_compare['order_total_value'].round(2))].shape

(2102, 115)

# customers 테이블과 join

In [27]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96264 entries, 0 to 96263
Data columns (total 2 columns):
customer_id           96264 non-null object
customer_unique_id    96264 non-null object
dtypes: object(2)
memory usage: 1.5+ MB


In [28]:
print(len(set(dataset2.customer_id)))
print(len(set(customers.customer_id)))
set(dataset2.customer_id) < set(customers.customer_id)

93000
96264


True

In [29]:
dataset3 = pd.merge(dataset2, customers, how = 'left', on = 'customer_id')

In [30]:
print(dataset2.shape)
print(dataset3.shape)

(93000, 113)
(93000, 114)


In [31]:
dataset3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 92999
Data columns (total 114 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
customer_id                      93000 non-null object
customer_city                    93000 non-null object
customer_state                   93000 non-null object
customer_zip_code_prefix         93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       9

In [32]:
dataset3.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2295
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             85243
review_comment_message           54108
review_creation_date                 0
review_answer_timestamp              0
product_category_name_english        0
installments_1           

# product_measures 테이블과 merge

In [33]:
print(len(set(dataset3.product_id)))
print(len(set(product_measures.product_id)))
print(product_measures.shape[0])
print( set(dataset3.product_id) < set(product_measures.product_id) )

23175
24439
24439
False


In [34]:
product_measures.isna().sum()

product_id           0
product_weight_g     0
product_length_cm    0
product_height_cm    0
product_width_cm     0
dtype: int64

In [35]:
dataset4 = pd.merge(dataset3, product_measures, how = 'left', on = 'product_id')

In [36]:
dataset4.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2295
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             85243
review_comment_message           54108
review_creation_date                 0
review_answer_timestamp              0
product_category_name_english        0
installments_1           

In [37]:
dataset4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 92999
Data columns (total 118 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
customer_id                      93000 non-null object
customer_city                    93000 non-null object
customer_state                   93000 non-null object
customer_zip_code_prefix         93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       9

# sellers 테이블과 merge

In [38]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98981 entries, 0 to 98980
Data columns (total 6 columns):
order_id                  98981 non-null object
product_id                98981 non-null object
seller_id                 98981 non-null object
seller_zip_code_prefix    98981 non-null object
seller_city               98981 non-null object
seller_state              98981 non-null object
dtypes: object(6)
memory usage: 4.5+ MB


In [39]:
sellers.columns

Index(['order_id', 'product_id', 'seller_id', 'seller_zip_code_prefix',
       'seller_city', 'seller_state'],
      dtype='object')

In [40]:
dataset4.columns

Index(['order_id', 'order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       ...
       'value_26', 'value_27', 'value_28', 'value_29', 'sequential',
       'customer_unique_id', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm'],
      dtype='object', length=118)

In [41]:
print(sellers[sellers.duplicated('product_id', keep=False)].sort_values('product_id').shape)
print(sellers[sellers.duplicated(subset=['product_id', 'seller_id'], keep=False)].sort_values('product_id').shape)

(85994, 6)
(84885, 6)


In [42]:
sellers.product_id.nunique()

24440

In [43]:
prod_seller = sellers.groupby(by = 'product_id')['seller_id'].apply(set).to_dict()
len(prod_seller)

24440

In [44]:
sellers_per_prod = [len(item) for item in prod_seller.values()]
sellers_per_prod

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


------> product_id에 복수의 seller_id가 존재함

In [45]:
pd.merge(dataset4, sellers, how = 'left', on = ['order_id', 'product_id']).isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2295
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             85243
review_comment_message           54108
review_creation_date                 0
review_answer_timestamp              0
product_category_name_english        0
installments_1           

In [46]:
dataset5 = pd.merge(dataset4, sellers, how = 'left', on = ['order_id', 'product_id'])

In [47]:
pd.set_option('display.max_categories', 100)

In [48]:
dataset5.head()

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_id,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name_english,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_13,installments_14,installments_15,installments_16,installments_17,installments_18,installments_19,installments_20,installments_21,installments_22,installments_23,installments_24,installments_25,installments_26,installments_27,installments_28,installments_29,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,payment_type_6,payment_type_7,payment_type_8,payment_type_9,payment_type_10,payment_type_11,payment_type_12,payment_type_13,payment_type_14,payment_type_15,payment_type_16,payment_type_17,payment_type_18,payment_type_19,payment_type_20,payment_type_21,payment_type_22,payment_type_23,payment_type_24,payment_type_25,payment_type_26,payment_type_27,payment_type_28,payment_type_29,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,value_16,value_17,value_18,value_19,value_20,value_21,value_22,value_23,value_24,value_25,value_26,value_27,value_28,value_29,sequential,customer_unique_id,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01,2017-02-01 02:41:21.549551,2017-03-15,2017-02-06 11:04:24.154259,297dcf281305aabfee256d313076af2b,Guaratuba,PR,832,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07,2017-02-09 02:37:37,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boleto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,363.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,a27e24da3a7dc34713304a30896a3c4e,900.0,25.0,12.0,38.0,ccc4bbb5f32a6ab2b7066a4130f114e3,803,curitiba,PR
1,8a723730400b508cbf47fbef4a76ec8e,delivered,60.00,20.91,1,1,2018-02-18 12:41:01,2018-02-18 13:46:36.606696,2018-03-14,2018-03-03 02:48:53.603256,f19376e0c54da9409485f7ef9556d367,Belo Horizonte,MG,308,38,1665,1,5858f45c20fde7d7e49af37a2166635a,8a723730400b508cbf47fbef4a76ec8e,5,NaN,muito bom cabelo fica lisinho,2018-03-04,2018-03-07 02:53:50,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit_card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,04533db2963aa82a202f214aeff155a6,2300.0,17.0,9.0,32.0,b56906f7fd1696e043f1bcce164c487b,198,echapora,SP
2,cf71b9b6b7ed12d72f541b0420023f19,delivered,79.99,10.94,1,1,2018-05-09 23:33:28,2018-05-11 02:57:52.916830,2018-05-24,2018-05-17 22:31:58.137648,12698ad9da6b62003811594de0ae297e,Cachoeiro de Itapemirim,ES,293,59,492,3,2b4609f8948be18874494203496bc318,cf71b9b6b7ed12d72f541b0420023f19,4,Bom,gostei,2018-05-18,2018-05-20 19:47:57,health_beauty,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boleto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.93,

In [49]:
dataset5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 92999
Data columns (total 122 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
customer_id                      93000 non-null object
customer_city                    93000 non-null object
customer_state                   93000 non-null object
customer_zip_code_prefix         93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       9

In [50]:
dataset5.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2295
customer_id                          0
customer_city                        0
customer_state                       0
customer_zip_code_prefix             0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_id                           0
review_id                            0
review_score                         0
review_comment_title             85243
review_comment_message           54108
review_creation_date                 0
review_answer_timestamp              0
product_category_name_english        0
installments_1           

# <<<<컬럼 순서 변경>>>>

In [51]:
dataset6 = dataset5.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 17, 24, 14, 15, 16, 114, 115, 116, 117, 113, 10, 12, 11, 13, 118, 121, 120, 119, 112, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 18, 19, 20, 21, 22, 23
]]
dataset6.head()
dataset6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 92999
Data columns (total 122 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
product_id                       93000 non-null object
product_category_name_english    93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       93000 non-null int64
product_photos_qty               93000 non-null int64
product_weight_g                 929

In [52]:
dataset6.isna().sum()

order_id                             0
order_status                         0
order_products_value                 0
order_freight_value                  0
order_items_qty                      0
order_sellers_qty                    0
order_purchase_timestamp             0
order_aproved_at                    18
order_estimated_delivery_date        0
order_delivered_customer_date     2295
product_id                           0
product_category_name_english        0
product_name_lenght                  0
product_description_lenght           0
product_photos_qty                   0
product_weight_g                    18
product_length_cm                   18
product_height_cm                   18
product_width_cm                    18
customer_unique_id                   0
customer_id                          0
customer_state                       0
customer_city                        0
customer_zip_code_prefix             0
seller_id                           17
seller_state             

# CSV로 저장

In [44]:
dataset6.to_csv('final_dataset2.csv', header = True, sep = ',', index = False, index_label = False)

# -------------------------------------------------------------------------------

* Unnamed: 0 와 id 컬럼을 제외한 모든 변수컬럼이 같은 경우가 존재함

# geolocation과 merge ( 나중에..... ㅠㅠ)

In [100]:
geolocation.zip_code_prefix.value_counts().describe()

count    851.000000
mean     379.572268
std      158.614428
min        1.000000
25%      251.500000
50%      500.000000
75%      500.000000
max      500.000000
Name: zip_code_prefix, dtype: float64

# classified (merge 불가)

In [306]:
classified2 = pd.merge(classified, translation, how = 'left', on = 'product_category_name').drop('product_category_name', axis = 1)
classified2

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
0,0,1,delivered,89.99,14.38,1,1,2017-08-30 11:41:01,2017-08-30 11:55:08.970352,2017-09-21,2017-09-08 20:35:27.276847,Belo Horizonte,MG,308,59,492,3,5,NaN,tudo certo,2017-09-09,2017-09-10 03:27:54,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,health_beauty
1,1,2,delivered,69.00,15.23,1,1,2017-09-26 09:13:36,2017-09-26 09:28:10.922048,2017-10-24,2017-09-29 21:13:04.984841,Pocos de Caldas,MG,377,50,679,4,5,NaN,o produto chegou antes de prazo parabens pela ...,2017-09-30,2017-10-03 05:34:20,3,0,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido,toys
2,2,3,delivered,99.80,15.86,2,4,2018-01-15 15:50:42,2018-01-17 07:29:56.419769,2018-02-05,2018-01-23 17:51:31.134866,Sao Jose dos Campos,SP,122,59,341,2,1,NaN,foi adquirido 6 itens desta loja onde até o mo...,2018-01-24,2018-02-02 17:42:43,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,garden_tools
3,3,4,delivered,87.00,12.74,1,1,2018-02-04 11:16:42,2018-02-06 05:31:50.990164,2018-03-13,2018-02-20 19:38:06.633080,Ribeirao Preto,SP,140,45,411,1,4,NaN,achei a entrega um pouco demorada,2018-02-21,2018-02-22 02:09:12,0,3,0,0,0,0,0,0,0,atrasado,problemas_de_entrega,computers_accessories
4,4,5,delivered,99.90,17.95,1,2,2017-12-07 11:58:42,2017-12-08 02:36:49.587515,2018-01-03,2017-12-19 22:33:18.952512,RIO DE JANEIRO,RJ,205,60,189,1,3,NaN,comprei 02 jogos de lencois de cetim e uma col...,2017-12-20,2017-12-23 04:17:03,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,bed_bath_table
5,5,6,delivered,39.99,0.15,1,6,2017-11-02 11:33:03,2017-11-04 08:05:48.050186,2017-11-23,2017-11-09 19:09:10.241896,Sao Paulo,SP,20,53,386,1,5,NaN,apesar de ser da mesma loja os produtos vieram...,2017-11-10,2017-11-13 22:28:12,0,0,0,0,0,1,1,1,0,NaN,NaN,fashion_bags_accessories
6,6,7,delivered,229.90,30.01,1,1,2017-11-26 23:58:24,2017-11-27 00:18:31.179577,2017-12-28,2017-12-19 14:13:52.655799,Floresta,PE,564,54,1120,8,2,NaN,não gostei de ter que ir retirar no correio qu...,2017-12-20,2017-12-21 12:44:54,0,0,0,0,0,0,3,0,0,outro_entrega,problemas_de_entrega,furniture_decor
7,7,8,delivered,164.90,8.41,1,1,2017-12-08 21:13:04,2017-12-08 21:32:36.905274,2017-12-27,2017-12-26 16:09:57.768207,SAO PAULO,SP,83,50,448,1,5,NaN,recomendo essa loja entrega antes do prazo,2017-12-27,2017-12-27 23:23:17,2,0,0,0,0,0,0,0,1,antes_prazo,satisfeito_com_pedido,watches_gifts
8,8,9,delivered,89.90,40.62,1,1,2017-07-04 16:12:36,2017-07-04 16:24:00.454971,2017-07-24,2017-07-10 16:35:53.500041,Juquitiba,SP,69,47,482,2,3,NaN,produto entregue dentro do prazo,2017-07-11,2017-07-14 14:42:17,1,0,0,0,0,0,0,0,2,satisfeito,satisfeito_com_pedido,luggage_accessories
9,9,10,delivered,99.90,19.94,1,1,2017-11-29 04:25:45,2017-12-01 11:32:01.170224,2017-12-29,2017-12-18 15:22:57.104615,Salvador,BA,403,60,189,1,4,NaN,adorei nota 10,2017-12-19,2017-12-21 20:10:55,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,bed_bath_table


* classified내에 중복된 내용이 있는지 확인 -> unique하게 만든다
* classified와 dataset를 연결하기 위해 어떤 컬럼들을사용해야 하는지 결정하기

In [129]:
pd.set_option('display.max_columns', None)

In [307]:
print(classified2.columns)
print(classified2.shape[0])

Index(['Unnamed: 0', 'id', 'order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'votes_before_estimate', 'votes_delayed',
       'votes_low_quality', 'votes_return', 'votes_not_as_anounced',
       'votes_partial_delivery', 'votes_other_delivery', 'votes_other_order',
       'votes_satisfied', 'most_voted_subclass', 'most_voted_class',
       'product_category_name_english'],
      dtype='object')
3584


### a) 완전히 동일한 row는 1개의 row만 남기고 삭제하기  (3584rows -> 3570rows)

#### 28개의 row는 14개의 row가 2번씩 duplicate되어 있어, 한개만 남겨주어야한다.

In [308]:
classified2.duplicated(subset = ['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'votes_before_estimate', 'votes_delayed', 'votes_low_quality',
       'votes_return', 'votes_not_as_anounced', 'votes_partial_delivery',
       'votes_other_delivery', 'votes_other_order', 'votes_satisfied',
       'most_voted_subclass', 'most_voted_class'], keep = False).sum()

28

In [222]:
pd.set_option('display.max_rows', 20)

In [309]:
classified2[classified2.duplicated(subset = ['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'votes_before_estimate', 'votes_delayed', 'votes_low_quality',
       'votes_return', 'votes_not_as_anounced', 'votes_partial_delivery',
       'votes_other_delivery', 'votes_other_order', 'votes_satisfied',
       'most_voted_subclass', 'most_voted_class'], keep = False)].sort_values(['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'votes_before_estimate', 'votes_delayed', 'votes_low_quality',
       'votes_return', 'votes_not_as_anounced', 'votes_partial_delivery',
       'votes_other_delivery', 'votes_other_order', 'votes_satisfied',
       'most_voted_subclass', 'most_voted_class'])

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
391,391,392,delivered,19.00,7.90,1,4,2017-12-14 20:30:29,2017-12-14 20:38:29.191463,2018-01-12,2017-12-23 14:33:14.894802,Pocos de Caldas,MG,377,58,237,1,1,NaN,fiz o pedido de 4 peças e só veio uma,2017-12-24,2017-12-26 19:30:59,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,auto
2296,2296,2297,delivered,19.00,7.90,1,4,2017-12-14 20:30:29,2017-12-14 20:38:29.191463,2018-01-12,2017-12-23 14:33:14.894802,Pocos de Caldas,MG,377,58,237,1,1,NaN,fiz o pedido de 4 peças e só veio uma,2017-12-24,2017-12-26 19:30:59,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,auto
1832,1832,1833,delivered,19.90,34.15,1,2,2017-06-18 14:01:13,2017-06-20 10:43:47.335041,2017-07-27,2017-07-21 18:46:41.449801,Sao Luis,MA,650,51,372,1,5,NaN,gostei dos produtos só precisa melhorar o valo...,2017-07-22,2017-07-23 16:14:54,0,0,0,0,0,0,1,1,1,NaN,NaN,furniture_decor
3351,3351,3352,delivered,19.90,34.15,1,2,2017-06-18 14:01:13,2017-06-20 10:43:47.335041,2017-07-27,2017-07-21 18:46:41.449801,Sao Luis,MA,650,51,372,1,5,NaN,gostei dos produtos só precisa melhorar o valo...,2017-07-22,2017-07-23 16:14:54,0,0,0,0,0,0,1,1,1,NaN,NaN,furniture_decor
1085,1085,1086,delivered,22.90,15.10,1,2,2017-05-29 21:46:58,2017-05-29 21:55:18.706403,2017-06-21,2017-06-06 19:14:35.279012,RIO DE JANEIRO,RJ,224,50,365,3,5,NaN,tudo certo,2017-06-08,2017-06-08 22:56:04,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,pet_shop
2160,2160,2161,delivered,22.90,15.10,1,2,2017-05-29 21:46:58,2017-05-29 21:55:18.706403,2017-06-21,2017-06-06 19:14:35.279012,RIO DE JANEIRO,RJ,224,50,365,3,5,NaN,tudo certo,2017-06-08,2017-06-08 22:56:04,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,pet_shop
880,880,881,delivered,24.90,11.85,1,2,2017-11-24 10:28:01,2017-11-25 04:31:14.355442,2017-12-14,2017-11-30 20:09:05.896428,Jumirim,SP,185,58,58,1,5,NaN,muito bom,2017-12-01,2017-12-01 18:39:41,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,bed_bath_table
1047,1047,1048,delivered,24.90,11.85,1,2,2017-11-24 10:28:01,2017-11-25 04:31:14.355442,2017-12-14,2017-11-30 20:09:05.896428,Jumirim,SP,185,58,58,1,5,NaN,muito bom,2017-12-01,2017-12-01 18:39:41,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,bed_bath_table
2422,2422,2423,delivered,39.90,6.67,1,10,2017-09-20 09:40:52,2017-09-20 09:55:23.919114,2017-11-09,2017-09-22 15:43:48.043802,Sao Paulo,SP,55,25,827,1,5,NaN,satisfeita,2017-09-23,2017-09-25 14:46:38,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,fashion_bags_accessories
3370,3370,3371,delivered,39.90,6.67,1,10,2017-09-20 09:40:52,2017-09-20 09:55:23.919114,2017-11-09,2017-09-22 15:43:48.043802,Sao Paulo,SP,55,25,827,1,5,NaN,satisfeita,2017-09-23,2017-09-25 14:46:38,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,fashion_bags_accessories


------- > 'Unnamed: 0' 와 'id' 컬럼을 제외한 모든 변수컬럼이 같은 경우가 존재함
이는 완전히 동일한 데이터이므로 삭제한다.

In [310]:
classified3 = classified2.drop_duplicates(subset = ['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'votes_before_estimate', 'votes_delayed', 'votes_low_quality',
       'votes_return', 'votes_not_as_anounced', 'votes_partial_delivery',
       'votes_other_delivery', 'votes_other_order', 'votes_satisfied',
       'most_voted_subclass', 'most_voted_class'], keep = 'first')

---------------------------------------------------------------------------------------------------------------------------

### b) order-status~ review_answer_timestamp까지는 동일하나, votes 부분이 다른 경우

#### 'most_voted_class'가 NaN인 케이스는 삭제해줌

In [352]:
classified3.duplicated(subset = ['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'], keep = False).sum()

16

In [353]:
classified3[classified3.duplicated(subset = ['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'], keep = False)].sort_values(['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty'])

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
506,506,507,delivered,48.90,17.60,1,1,2017-09-15 10:24:16,2017-09-15 10:50:23.903902,2017-10-05,2017-09-26 19:22:26.483622,Volta Redonda,RJ,272,41,596,1,5,NaN,na primeira escova o produto pesa mas depois q...,2017-09-27,2017-10-04 13:33:55,0,0,0,0,0,0,0,0,2,satisfeito,satisfeito_com_pedido,health_beauty
3069,3069,3070,delivered,48.90,17.60,1,1,2017-09-15 10:24:16,2017-09-15 10:50:23.903902,2017-10-05,2017-09-26 19:22:26.483622,Volta Redonda,RJ,272,41,596,1,5,NaN,na primeira escova o produto pesa mas depois q...,2017-09-27,2017-10-04 13:33:55,0,0,0,0,0,0,0,0,1,NaN,NaN,health_beauty
2862,2862,2863,delivered,72.90,18.04,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN,furniture_decor
2872,2872,2873,delivered,72.90,18.04,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega,furniture_decor
2434,2434,2435,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN,furniture_decor
2631,2631,2632,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega,furniture_decor
2240,2240,2241,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,60,471,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega,furniture_decor
2550,2550,2551,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,60,471,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN,furniture_decor
2614,2614,2615,delivered,74.25,53.76,3,1,2017-08-16 13:10:41,2017-08-16 13:30:18.084442,2017-09-18,2017-09-05 17:00:13.936722,Alto Boa Vista,MT,786,56,297,2,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN,furniture_decor
3513,3513,3514,delivered,74.25,53.76,3,1,2017-08-16 13:10:41,2017-08-16 13:30:18.084442,2017-09-18,2017-09-05 17:00:13.936722,Alto Boa Vista,MT,786,56,297,2,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega,furniture_decor


In [354]:
print(classified2.shape[0])
print(classified3.shape[0])
print(classified2.isna().sum())

3584
3570
Unnamed: 0                         0
id                                 0
order_status                       0
order_products_value               0
order_freight_value                0
order_items_qty                    0
order_sellers_qty                  0
order_purchase_timestamp           0
order_aproved_at                   0
order_estimated_delivery_date      0
                                ... 
votes_low_quality                  0
votes_return                       0
votes_not_as_anounced              0
votes_partial_delivery             0
votes_other_delivery               0
votes_other_order                  0
votes_satisfied                    0
most_voted_subclass              171
most_voted_class                 171
product_category_name_english      0
Length: 34, dtype: int64


In [355]:
index_temp = classified3[classified3.duplicated(subset = ['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'], keep = False)].sort_values(['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty']).index
print(index_temp)
index_temp2 = [index for index, value in classified3.loc[index_temp, :]['most_voted_class'].isnull().iteritems() if value == True]
print(index_temp2)
classified4 = classified3.drop(labels = index_temp2)
classified4

Int64Index([ 506, 3069, 2862, 2872, 2434, 2631, 2240, 2550, 2614, 3513, 2065,
            2090, 1109, 2725,  874, 1550],
           dtype='int64')
[3069, 2862, 2434, 2550, 2614, 2065, 2725, 1550]


,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
0,0,1,delivered,89.99,14.38,1,1,2017-08-30 11:41:01,2017-08-30 11:55:08.970352,2017-09-21,2017-09-08 20:35:27.276847,Belo Horizonte,MG,308,59,492,3,5,NaN,tudo certo,2017-09-09,2017-09-10 03:27:54,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,health_beauty
1,1,2,delivered,69.00,15.23,1,1,2017-09-26 09:13:36,2017-09-26 09:28:10.922048,2017-10-24,2017-09-29 21:13:04.984841,Pocos de Caldas,MG,377,50,679,4,5,NaN,o produto chegou antes de prazo parabens pela ...,2017-09-30,2017-10-03 05:34:20,3,0,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido,toys
2,2,3,delivered,99.80,15.86,2,4,2018-01-15 15:50:42,2018-01-17 07:29:56.419769,2018-02-05,2018-01-23 17:51:31.134866,Sao Jose dos Campos,SP,122,59,341,2,1,NaN,foi adquirido 6 itens desta loja onde até o mo...,2018-01-24,2018-02-02 17:42:43,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,garden_tools
3,3,4,delivered,87.00,12.74,1,1,2018-02-04 11:16:42,2018-02-06 05:31:50.990164,2018-03-13,2018-02-20 19:38:06.633080,Ribeirao Preto,SP,140,45,411,1,4,NaN,achei a entrega um pouco demorada,2018-02-21,2018-02-22 02:09:12,0,3,0,0,0,0,0,0,0,atrasado,problemas_de_entrega,computers_accessories
4,4,5,delivered,99.90,17.95,1,2,2017-12-07 11:58:42,2017-12-08 02:36:49.587515,2018-01-03,2017-12-19 22:33:18.952512,RIO DE JANEIRO,RJ,205,60,189,1,3,NaN,comprei 02 jogos de lencois de cetim e uma col...,2017-12-20,2017-12-23 04:17:03,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,bed_bath_table
5,5,6,delivered,39.99,0.15,1,6,2017-11-02 11:33:03,2017-11-04 08:05:48.050186,2017-11-23,2017-11-09 19:09:10.241896,Sao Paulo,SP,20,53,386,1,5,NaN,apesar de ser da mesma loja os produtos vieram...,2017-11-10,2017-11-13 22:28:12,0,0,0,0,0,1,1,1,0,NaN,NaN,fashion_bags_accessories
6,6,7,delivered,229.90,30.01,1,1,2017-11-26 23:58:24,2017-11-27 00:18:31.179577,2017-12-28,2017-12-19 14:13:52.655799,Floresta,PE,564,54,1120,8,2,NaN,não gostei de ter que ir retirar no correio qu...,2017-12-20,2017-12-21 12:44:54,0,0,0,0,0,0,3,0,0,outro_entrega,problemas_de_entrega,furniture_decor
7,7,8,delivered,164.90,8.41,1,1,2017-12-08 21:13:04,2017-12-08 21:32:36.905274,2017-12-27,2017-12-26 16:09:57.768207,SAO PAULO,SP,83,50,448,1,5,NaN,recomendo essa loja entrega antes do prazo,2017-12-27,2017-12-27 23:23:17,2,0,0,0,0,0,0,0,1,antes_prazo,satisfeito_com_pedido,watches_gifts
8,8,9,delivered,89.90,40.62,1,1,2017-07-04 16:12:36,2017-07-04 16:24:00.454971,2017-07-24,2017-07-10 16:35:53.500041,Juquitiba,SP,69,47,482,2,3,NaN,produto entregue dentro do prazo,2017-07-11,2017-07-14 14:42:17,1,0,0,0,0,0,0,0,2,satisfeito,satisfeito_com_pedido,luggage_accessories
9,9,10,delivered,99.90,19.94,1,1,2017-11-29 04:25:45,2017-12-01 11:32:01.170224,2017-12-29,2017-12-18 15:22:57.104615,Salvador,BA,403,60,189,1,4,NaN,adorei nota 10,2017-12-19,2017-12-21 20:10:55,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,bed_bath_table


In [356]:
classified4[classified4.duplicated(subset = ['order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_category_name_english', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'], keep = False)]

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english


-------------------------------------------------------------------------------------------------------------------

In [320]:
print(classified4.isna().sum())

Unnamed: 0                         0
id                                 0
order_status                       0
order_products_value               0
order_freight_value                0
order_items_qty                    0
order_sellers_qty                  0
order_purchase_timestamp           0
order_aproved_at                   0
order_estimated_delivery_date      0
                                ... 
votes_low_quality                  0
votes_return                       0
votes_not_as_anounced              0
votes_partial_delivery             0
votes_other_delivery               0
votes_other_order                  0
votes_satisfied                    0
most_voted_subclass              162
most_voted_class                 162
product_category_name_english      0
Length: 34, dtype: int64


### c) order_status ~ review_answer_timestamp와 dataset을 연결시킬수 있는지 확인

In [368]:
classified4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3562 entries, 0 to 3583
Data columns (total 34 columns):
Unnamed: 0                       3562 non-null int64
id                               3562 non-null int64
order_status                     3562 non-null object
order_products_value             3562 non-null float64
order_freight_value              3562 non-null float64
order_items_qty                  3562 non-null int64
order_sellers_qty                3562 non-null int64
order_purchase_timestamp         3562 non-null datetime64[ns]
order_aproved_at                 3562 non-null datetime64[ns]
order_estimated_delivery_date    3562 non-null datetime64[ns]
order_delivered_customer_date    3445 non-null datetime64[ns]
customer_city                    3562 non-null object
customer_state                   3562 non-null object
customer_zip_code_prefix         3562 non-null object
product_name_lenght              3562 non-null int64
product_description_lenght       3562 non-null int64
p

In [328]:
classified4.columns

Index(['Unnamed: 0', 'id', 'order_status', 'order_products_value',
       'order_freight_value', 'order_items_qty', 'order_sellers_qty',
       'order_purchase_timestamp', 'order_aproved_at',
       'order_estimated_delivery_date', 'order_delivered_customer_date',
       'customer_city', 'customer_state', 'customer_zip_code_prefix',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'votes_before_estimate', 'votes_delayed',
       'votes_low_quality', 'votes_return', 'votes_not_as_anounced',
       'votes_partial_delivery', 'votes_other_delivery', 'votes_other_order',
       'votes_satisfied', 'most_voted_subclass', 'most_voted_class',
       'product_category_name_english'],
      dtype='object')

In [336]:
classified4.duplicated(['order_products_value', 'order_freight_value', 'order_items_qty'
                        'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'], keep = False).sum()

0

In [380]:
dataset5.duplicated(['order_products_value', 'order_freight_value', 'order_items_qty',
                        'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'], keep=False).sum()

199

In [382]:
dataset5[dataset5.duplicated(['order_products_value', 'order_freight_value', 'order_items_qty',
                        'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'], keep=False)].sort_values(['order_products_value', 'order_freight_value', 'order_items_qty',
                        'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'])

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_id,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name_english,customer_unique_id,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_id,seller_zip_code_prefix,seller_city,seller_state
43934,3990f96693d321ac142fff312bf3706a,delivered,7.00,3.33,1,6,2018-07-08 16:13:13,2018-07-08 16:25:33.555019,2018-08-15,2018-07-11 11:28:38.831241,6c44a903274653cddb1df3bcb05ac71f,Curitiba,PR,806,30,138,1,a25583531530c0913ea4dee2c5c73685,3990f96693d321ac142fff312bf3706a,5,NaN,NaN,2018-07-12,2018-07-12 21:13:57,perfumery,0172d503d0bcefbc1fc8ef0e16ee516d,150.0,20.0,7.0,20.0,4e06067cc08b3f41d837768d392c3ee3,326,betim,MG
45091,3990f96693d321ac142fff312bf3706a,delivered,7.00,3.33,1,6,2018-07-08 16:13:13,2018-07-08 16:25:33.555019,2018-08-15,2018-07-11 11:28:38.831241,6c44a903274653cddb1df3bcb05ac71f,Curitiba,PR,806,30,138,1,274d4e727ddbeb7c392ea2e551e3a77d,3990f96693d321ac142fff312bf3706a,5,NaN,NaN,2018-07-12,2018-07-12 21:13:57,perfumery,0172d503d0bcefbc1fc8ef0e16ee516d,100.0,20.0,7.0,20.0,4e06067cc08b3f41d837768d392c3ee3,326,betim,MG
45922,3990f96693d321ac142fff312bf3706a,delivered,7.00,3.33,1,6,2018-07-08 16:13:13,2018-07-08 16:25:33.555019,2018-08-15,2018-07-11 11:28:38.831241,6c44a903274653cddb1df3bcb05ac71f,Curitiba,PR,806,30,138,1,4650fae75d852ddfada4751f6f02481a,3990f96693d321ac142fff312bf3706a,5,NaN,NaN,2018-07-12,2018-07-12 21:13:57,perfumery,0172d503d0bcefbc1fc8ef0e16ee516d,100.0,20.0,5.0,20.0,4e06067cc08b3f41d837768d392c3ee3,326,betim,MG
43075,5adea766ca8f3521f957b27154b09761,delivered,7.00,15.23,1,2,2018-07-28 20:32:43,2018-08-02 23:30:35.309790,2018-08-15,2018-08-09 14:16:41.926856,2cc81b7f51e08b9f200fff0fc2eec98d,Sao Paulo,SP,049,30,138,1,e53197022fd9f554e01b9cf4f3c9f249,5adea766ca8f3521f957b27154b09761,5,NaN,NaN,2018-08-17,2018-08-17 15:39:22,perfumery,ca7c08ecfc023a391f5f077eb943e8a8,150.0,20.0,5.0,20.0,4e06067cc08b3f41d837768d392c3ee3,326,betim,MG
43726,5adea766ca8f3521f957b27154b09761,delivered,7.00,15.23,1,2,2018-07-28 20:32:43,2018-08-02 23:30:35.309790,2018-08-15,2018-08-09 14:16:41.926856,2cc81b7f51e08b9f200fff0fc2eec98d,Sao Paulo,SP,049,30,138,1,3c44881c6f1115b3a16f8281bb2a83ff,5adea766ca8f3521f957b27154b09761,5,NaN,NaN,2018-08-17,2018-08-17 15:39:22,perfumery,ca7c08ecfc023a391f5f077eb943e8a8,100.0,20.0,7.0,20.0,4e06067cc08b3f41d837768d392c3ee3,326,betim,MG
27352,f04396b769e9563e83456389c7d8f043,delivered,7.80,1.80,1,10,2017-04-19 13:55:27,2017-04-20 02:55:28.840045,2017-05-29,2017-05-03 15:33:57.317741,0fb0dffa0163450892c5aea3d5c62557,Belo Horizonte,MG,312,50,608,2,7e5c371cb71a63fe72051e4263d826f1,f04396b769e9563e83456389c7d8f043,5,NaN,chegou antes do prazo,2017-05-04,2017-05-07 04:13:59,furniture_decor,1b890601ddf7e9913d2c05d2bebedc5a,80.0,25.0,4.0,25.0,5c853bb56f70f4d14218944bae111d7a,067,cotia,SP
30533,f04396b769e9563e83456389c7d8f043,delivered,7.80,1.80,1,10,2017-04-19 13:55:27,2017-04-20 02:55:28.840045,2017-05-29,2017-05-03 15:33:57.317741,0fb0dffa0163450892c5aea3d5c62557,Belo Horizonte,MG,312,50,608,2,7b31263d3ac685437d34ea6e07fd1503,f04396b769e9563e83456389c7d8f043,5,NaN,chegou antes do prazo,2017-05-04,2017-05-07 04:13:59,furniture_decor,1b890601ddf7e9913d2c05d2bebedc5a,500.0,25.0,4.0,25.0,5c853bb56f70f4d14218944bae111d7a,067,cotia,SP
31787,f4fdb235ce7c1f7d3eb1b2fdd3a4f610,delivered,17.90,15.10,1,2,2017-10-27 12:50:10,2017-10-27 13:07:28.702273,2017-11-14,2017-11-07 20:18:42.565966,841583c9439180dd784a95ad9498e34a,Belo Horizonte,MG,306,56,646,1,414c53fa9c5cc4a0390682aeb2e25ec1,f4fdb235ce7c1f7d3eb1b2fdd3a4f610,5,NaN,NaN,2017-11-08,2017-11-09 00:47:19,furniture_decor,c6fc7bf59092

In [376]:
#function = lambda x : if x.isna() : str(x)
def function(x) :
    if x is None : return 'nan'
    else: return str(x)
    
################################################################################################################################
df = classified4.loc[:,['order_products_value', 'order_freight_value',
                        'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp']]
ex = []
for index, row in df.iterrows():
    ex.append('-'.join(row.map(function)))
    

###############################################################################################################################
df2 = dataset5.loc[:,['order_products_value', 'order_freight_value',
                        'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp']]
ex2 = []
for(in)
#classified4['review_comment_title'].map(function)


['89.99-14.38-health_beauty-59-492-3-2017-09-08 20:35:27.276847-5-nan-tudo certo-2017-09-09 00:00:00-2017-09-10 03:27:54',
 '69.0-15.23-toys-50-679-4-2017-09-29 21:13:04.984841-5-nan-o produto chegou antes de prazo parabens pela pontualidade-2017-09-30 00:00:00-2017-10-03 05:34:20',
 '99.8-15.86-garden_tools-59-341-2-2018-01-23 17:51:31.134866-1-nan-foi adquirido 6 itens desta loja onde até o momento não foi entregue um 1 unidade pendente meia lua meia bola esfera alumínio 34cm vermelho solicito posicionamento da empresa para recebimento-2018-01-24 00:00:00-2018-02-02 17:42:43',
 '87.0-12.74-computers_accessories-45-411-1-2018-02-20 19:38:06.633080-4-nan-achei a entrega um pouco demorada-2018-02-21 00:00:00-2018-02-22 02:09:12',
 '99.9-17.95-bed_bath_table-60-189-1-2017-12-19 22:33:18.952512-3-nan-comprei 02 jogos de lencois de cetim e uma colcha mas até agora só recebi o jogo de lencol cor cobre o que houve-2017-12-20 00:00:00-2017-12-23 04:17:03',
 '39.99-0.15-fashion_bags_accessorie

In [324]:
classified4.duplicated(['review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'], keep = False).sum()

803

In [335]:
classified4[classified4.duplicated(['order_products_value', 'order_freight_value',
                        'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'], keep = False)]. sort_values(['order_products_value', 'order_freight_value',
                        'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'])

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
2580,2580,2581,delivered,129.34,17.82,1,2,2018-03-20 17:23:39,2018-03-22 02:50:09.481615,2018-04-06,2018-03-28 20:06:36.709910,Paulinia,SP,131,58,560,1,3,NaN,então gostei mais veio diferente do que eu ped...,2018-03-30,2018-04-05 05:08:02,0,0,0,0,5,0,0,0,1,diferente_do_anunciado,problemas_de_qualidade,furniture_decor
3065,3065,3066,delivered,129.34,17.82,1,2,2018-03-20 17:23:39,2018-03-22 02:50:09.481615,2018-04-06,2018-03-28 20:06:36.709910,Paulinia,SP,131,58,560,1,3,NaN,então gostei mais veio diferente do que eu ped...,2018-03-30,2018-04-05 05:08:02,0,0,0,0,5,0,0,0,1,diferente_do_anunciado,problemas_de_qualidade,bed_bath_table


In [360]:
classified4.review_comment_title = classified4.review_comment_title.astype(object)
classified4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3562 entries, 0 to 3583
Data columns (total 34 columns):
Unnamed: 0                       3562 non-null int64
id                               3562 non-null int64
order_status                     3562 non-null object
order_products_value             3562 non-null float64
order_freight_value              3562 non-null float64
order_items_qty                  3562 non-null int64
order_sellers_qty                3562 non-null int64
order_purchase_timestamp         3562 non-null datetime64[ns]
order_aproved_at                 3562 non-null datetime64[ns]
order_estimated_delivery_date    3562 non-null datetime64[ns]
order_delivered_customer_date    3445 non-null datetime64[ns]
customer_city                    3562 non-null object
customer_state                   3562 non-null object
customer_zip_code_prefix         3562 non-null object
product_name_lenght              3562 non-null int64
product_description_lenght       3562 non-null int64
p

In [361]:
pd.merge(dataset5, classified4, how = 'left' , on = ['order_products_value', 'order_freight_value',
                        'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                        'order_delivered_customer_date',
                        'review_score', 'review_comment_title', 'review_comment_message',
                        'review_creation_date', 'review_answer_timestamp'])

,order_id,order_status_x,order_products_value,order_freight_value,order_items_qty_x,order_sellers_qty_x,order_purchase_timestamp_x,order_aproved_at_x,order_estimated_delivery_date_x,order_delivered_customer_date,customer_id,customer_city_x,customer_state_x,customer_zip_code_prefix_x,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name_english,customer_unique_id,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_id,seller_zip_code_prefix,seller_city,seller_state,Unnamed: 0,id,order_status_y,order_items_qty_y,order_sellers_qty_y,order_purchase_timestamp_y,order_aproved_at_y,order_estimated_delivery_date_y,customer_city_y,customer_state_y,customer_zip_code_prefix_y,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01,2017-02-01 02:41:21.549551,2017-03-15,2017-02-06 11:04:24.154259,297dcf281305aabfee256d313076af2b,Guaratuba,PR,832,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07,2017-02-09 02:37:37,health_beauty,a27e24da3a7dc34713304a30896a3c4e,900.0,25.0,12.0,38.0,ccc4bbb5f32a6ab2b7066a4130f114e3,803,curitiba,PR,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e98077a0d199a25a40eab3b14cc230d4,delivered,39.99,15.23,1,2,2018-04-17 13:25:36,2018-04-18 13:31:35.349522,2018-05-10,2018-04-27 20:56:32.787589,5161d5feec98ade754f67746caf8e0c9,BRASILIA,DF,724,51,405,2,190d9562bfbe9d3ed876c2ac6f2f5894,e98077a0d199a25a40eab3b14cc230d4,5,NaN,NaN,2018-04-28,2018-04-29 21:07:53,health_beauty,c96f477f96ccbb7cb476a48ea6840098,150.0,20.0,20.0,20.0,0ac4201fda2c68ebc0e47cb9423cf3c9,033,sao paulo,SP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8a723730400b508cbf47fbef4a76ec8e,delivered,60.00,20.91,1,1,2018-02-18 12:41:01,2018-02-18 13:46:36.606696,2018-03-14,2018-03-03 02:48:53.603256,f19376e0c54da9409485f7ef9556d367,Belo Horizonte,MG,308,38,1665,1,5858f45c20fde7d7e49af37a2166635a,8a723730400b508cbf47fbef4a76ec8e,5,NaN,muito bom cabelo fica lisinho,2018-03-04,2018-03-07 02:53:50,health_beauty,04533db2963aa82a202f214aeff155a6,2300.0,17.0,9.0,32.0,b56906f7fd1696e043f1bcce164c487b,198,echapora,SP,2901.0,2902.0,delivered,1.0,1.0,2018-02-18 12:41:01,2018-02-18 13:46:36.606696,2018-03-14,Belo Horizonte,MG,308,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,satisfeito,satisfeito_com_pedido
3,cf71b9b6b7ed12d72f541b0420023f19,delivered,79.99,10.94,1,1,2018-05-09 23:33:28,2018-05-11 02:57:52.916830,2018-05-24,2018-05-17 22:31:58.137648,12698ad9da6b62003811594de0ae297e,Cachoeiro de Itapemirim,ES,293,59,492,3,2b4609f8948be18874494203496bc318,cf71b9b6b7ed12d72f541b0420023f19,4,Bom,gostei,2018-05-18,2018-05-20 19:47:57,health_beauty,0e37bd1acb9088c21332942d2a334cf7,250.0,22.0,10.0,18.0,cc419e0650a3c5ba77189a1882b7556a,090,santo andre,SP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,93ef9253c47e7dfc633c01f728488cb1,delivered,58.99,11.73,1,1,2017-10-26 09:07:04,2017-10-27 09:15:12.032978,2017-11-09,2017-10-31 16:09:47.272654,ba21875fac3c41b6ac433113cf862e55,Maua,SP,093,59,492,2,7c1bd920dbdf22470b68bde975dd3ccf,93ef9253c47e7dfc633c01f728488cb1,4,NaN,NaN,2017-11-01,2017-11-03 20:37:36,health_beauty,3e25ca97251fcdc2ee6597de4bb15f2c,200.0,22.0,10.0,18.0,cc419e0650a3c5ba77189a1882b7556a,090,santo andre,SP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,35bc592a16d6706d14d29f07ed1d13c1,delivered,639.00,21.40,1,1,2018-08-11 15:41:42,2018-08-11 15:55:15.569411,2018-08-28,2018-08-22 17:18:51.759036,b88a511344b8b49a02bca030ace61ef6,RIO DE JANEIRO,RJ,227,59,2141,1,

In [273]:
dataset5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98014 entries, 0 to 98013
Data columns (total 34 columns):
order_id                         98014 non-null object
order_status                     98014 non-null object
order_products_value             98014 non-null float64
order_freight_value              98014 non-null float64
order_items_qty                  98014 non-null int64
order_sellers_qty                98014 non-null int64
order_purchase_timestamp         98014 non-null datetime64[ns]
order_aproved_at                 97996 non-null datetime64[ns]
order_estimated_delivery_date    98014 non-null datetime64[ns]
order_delivered_customer_date    95659 non-null datetime64[ns]
customer_id                      98014 non-null object
customer_city                    98014 non-null object
customer_state                   98014 non-null object
customer_zip_code_prefix         98014 non-null object
product_name_lenght              98014 non-null int64
product_description_lenght       98

In [322]:
classified4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3562 entries, 0 to 3583
Data columns (total 34 columns):
Unnamed: 0                       3562 non-null int64
id                               3562 non-null int64
order_status                     3562 non-null object
order_products_value             3562 non-null float64
order_freight_value              3562 non-null float64
order_items_qty                  3562 non-null int64
order_sellers_qty                3562 non-null int64
order_purchase_timestamp         3562 non-null datetime64[ns]
order_aproved_at                 3562 non-null datetime64[ns]
order_estimated_delivery_date    3562 non-null datetime64[ns]
order_delivered_customer_date    3445 non-null datetime64[ns]
customer_city                    3562 non-null object
customer_state                   3562 non-null object
customer_zip_code_prefix         3562 non-null object
product_name_lenght              3562 non-null int64
product_description_lenght       3562 non-null int64
p

In [239]:
classified.iloc[index_temp, :]

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class
506,506,507,delivered,48.90,17.60,1,1,2017-09-15 10:24:16,2017-09-15 10:50:23.903902,2017-10-05,2017-09-26 19:22:26.483622,Volta Redonda,RJ,272,beleza_saude,41,596,1,5,NaN,na primeira escova o produto pesa mas depois q...,2017-09-27,2017-10-04 13:33:55,0,0,0,0,0,0,0,0,2,satisfeito,satisfeito_com_pedido
3069,3069,3070,delivered,48.90,17.60,1,1,2017-09-15 10:24:16,2017-09-15 10:50:23.903902,2017-10-05,2017-09-26 19:22:26.483622,Volta Redonda,RJ,272,beleza_saude,41,596,1,5,NaN,na primeira escova o produto pesa mas depois q...,2017-09-27,2017-10-04 13:33:55,0,0,0,0,0,0,0,0,1,NaN,NaN
2862,2862,2863,delivered,72.90,18.04,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,moveis_decoracao,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN
2872,2872,2873,delivered,72.90,18.04,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,moveis_decoracao,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega
2434,2434,2435,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,moveis_decoracao,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN
2631,2631,2632,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,moveis_decoracao,60,458,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega
2240,2240,2241,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,moveis_decoracao,60,471,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega
2550,2550,2551,delivered,72.90,18.10,1,3,2017-08-16 13:10:42,2017-08-16 13:30:20.049901,2017-09-26,2017-09-05 18:43:15.566530,Alto Boa Vista,MT,786,moveis_decoracao,60,471,1,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN
2614,2614,2615,delivered,74.25,53.76,3,1,2017-08-16 13:10:41,2017-08-16 13:30:18.084442,2017-09-18,2017-09-05 17:00:13.936722,Alto Boa Vista,MT,786,moveis_decoracao,56,297,2,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,1,0,0,0,NaN,NaN
3513,3513,3514,delivered,74.25,53.76,3,1,2017-08-16 13:10:41,2017-08-16 13:30:18.084442,2017-09-18,2017-09-05 17:00:13.936722,Alto Boa Vista,MT,786,moveis_decoracao,56,297,2,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06,2017-09-11 15:46:19,0,0,0,0,0,2,0,0,0,entrega_parcial,problemas_de_entrega


In [121]:
classified.sort_values(by = 'review_comment_message').loc[:, ['review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']]

,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1109,5,NaN,0,2017-11-08 00:00:00.000000,2017-11-10 09:53:00+00:00
2725,5,NaN,0,2017-11-08 00:00:00.000000,2017-11-10 09:53:00+00:00
2240,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2862,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
3513,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2872,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2090,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2065,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2434,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2550,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00


In [121]:
classified.sort_values(by = 'review_comment_message').loc[:, ['review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']]

,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1109,5,NaN,0,2017-11-08 00:00:00.000000,2017-11-10 09:53:00+00:00
2725,5,NaN,0,2017-11-08 00:00:00.000000,2017-11-10 09:53:00+00:00
2240,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2862,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
3513,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2872,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2090,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2065,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2434,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00
2550,2,NaN,01 00000089 pedido referente a três cortinas p...,2017-09-06 00:00:00.000000,2017-09-11 15:46:19+00:00


In [104]:
classified.loc[:, ['review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']]

,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,5,NaN,tudo certo,2017-09-09 00:00:00.000000,2017-09-10 03:27:54+00:00
1,5,NaN,o produto chegou antes de prazo parabens pela ...,2017-09-30 00:00:00.000000,2017-10-03 05:34:20+00:00
2,1,NaN,foi adquirido 6 itens desta loja onde até o mo...,2018-01-24 00:00:00.000000,2018-02-02 17:42:43+00:00
3,4,NaN,achei a entrega um pouco demorada,2018-02-21 00:00:00.000000,2018-02-22 02:09:12+00:00
4,3,NaN,comprei 02 jogos de lencois de cetim e uma col...,2017-12-20 00:00:00.000000,2017-12-23 04:17:03+00:00
5,5,NaN,apesar de ser da mesma loja os produtos vieram...,2017-11-10 00:00:00.000000,2017-11-13 22:28:12+00:00
6,2,NaN,não gostei de ter que ir retirar no correio qu...,2017-12-20 00:00:00.000000,2017-12-21 12:44:54+00:00
7,5,NaN,recomendo essa loja entrega antes do prazo,2017-12-27 00:00:00.000000,2017-12-27 23:23:17+00:00
8,3,NaN,produto entregue dentro do prazo,2017-07-11 00:00:00.000000,2017-07-14 14:42:17+00:00
9,4,NaN,adorei nota 10,2017-12-19 00:00:00.000000,2017-12-21 20:10:55+00:00


In [107]:
classified['review_comment_message'].isna().sum()

0

In [109]:
classified['review_comment_message'].nunique()

2812

In [117]:
class_key = str(classified['review_score']) + classified['review_comment_message'] + classified['review_creation_date'] + classified['review_answer_timestamp']
class_key.nunique()

3114

In [118]:
classified['review_comment_message'].nunique()

2812

In [119]:
classified['review_comment_message'].value_counts()

muito bom                                                                                                                                                                                               48
bom                                                                                                                                                                                                     30
recomendo                                                                                                                                                                                               26
excelente                                                                                                                                                                                               19
otimo                                                                                                                                                                                       

In [116]:
classified.duplicated(keep=False).sum()

0